In [2]:
import polib

# Load the original file
original_file = polib.pofile('futurecoder_English.po')

# Create a new file with only translations
cleaned_file = polib.POFile()

# Iterate over the original file's entries and add only the translations to the new file
for entry in original_file:
    if entry.msgstr:
        new_entry = polib.POEntry(
            msgid=entry.msgid,
            msgstr=entry.msgstr
        )
        cleaned_file.append(new_entry)

# Save the new file
cleaned_file.save('futurecoder_English_cleaned.po')


In [4]:
import os
import polib

# Load the cleaned file
cleaned_file = polib.pofile('futurecoder_English_cleaned.po')

# Create a dictionary to store entries by category
entries_by_category = {}

# Iterate over the cleaned file's entries and add them to the dictionary by category
for entry in cleaned_file:
    category = entry.msgid.split('.')[0]
    if category not in entries_by_category:
        entries_by_category[category] = polib.POFile()
    entries_by_category[category].append(entry)

# Save each category to its own po file in the categories directory
if not os.path.exists('categories'):
    os.makedirs('categories')
for category, entries in entries_by_category.items():
    entries.save(f'categories/{category}.po')


In [6]:
import os
import polib

# Load the cleaned file
cleaned_file = polib.pofile('futurecoder_English_cleaned.po')

# Create a dictionary to store entries by page
entries_by_page = {}
order = []
# Iterate over the cleaned file's entries and add them to the dictionary by page
for entry in cleaned_file:
    if entry.msgid.startswith('pages.'):
        page = entry.msgid.split('.')[1]
        if page not in entries_by_page:
            entries_by_page[page] = polib.POFile()
            order.append(page)
        entries_by_page[page].append(entry)

# Save each page to its own po file in the pages directory
if not os.path.exists('pages'):
    os.makedirs('pages')
for page, entries in entries_by_page.items():
    page_order = order.index(page)
    entries.save(f'pages/{page_order}_{page}.po')


In [7]:
from openai import OpenAI

def translate(text, dry_run=False):
    if dry_run:
        return text
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "translate to polish"},
            {"role": "user", "content": text},
        ],
    )
    return response.choices[0].message.content


In [10]:
import os
import polib
from typing import List
from pathlib import Path
# 1. Import the necessary modules

# 2. Define a function to read a file as a string
def read_file_as_string(file_path: str) -> str:
    with open(file_path, 'r') as f:
        return f.read()

# 3. Define a function to write a string to a file
def write_string_to_file(file_path: str, content: str) -> None:
    with open(file_path, 'w') as f:
        f.write(content)

# 4. Define a function to translate a string using the translate function from CELL INDEX 3
def translate_string(text: str, dry_run: bool = False) -> str:
    if dry_run:
        return text
    return translate(text)

# 5. Define a function to iterate over all the po files in the pages and categories directories, read each file as a string, pass it to the translate function with dry_run=True, and write the result to a new file in the translated directory with the same structure as the original file
def translate_files() -> None:
    # Create the translated directory if it doesn't exist
    if not os.path.exists('translated'):
        os.makedirs('translated')

    # Iterate over the categories and pages directories
    # for directory in ['categories', 'pages']:
    for directory in ['pages']:
        # Create the corresponding directory in the translated directory if it doesn't exist
        if not os.path.exists(f'translated/{directory}'):
            os.makedirs(f'translated/{directory}')

        # Iterate over the po files in the directory
        for file_path in Path(directory).rglob('*.po'):
            print(f'Translating {file_path}...')
            # Read the file as a string
            original_content = read_file_as_string(str(file_path))

            # Translate the string
            translated_content = translate_string(original_content, dry_run=False)

            # Write the translated string to a new file in the translated directory with the same structure as the original file
            translated_file_path = f'translated/{directory}/{file_path.relative_to(directory)}'
            write_string_to_file(str(translated_file_path), translated_content)

# 6. Call the function from step 5
translate_files()


Translating pages/22_BuildingNewLists.po...
Translating pages/25_CallingFunctionsTerminology.po...
Translating pages/12_BuildingUpStringsExercises.po...
Translating pages/35_IntroducingNestedLoops.po...
Translating pages/20_OtherComparisonOperators.po...
Translating pages/19_IntroducingElif.po...
Translating pages/7_StoringCalculationsInVariables.po...
Translating pages/47_CombiningAndAndOr.po...
Translating pages/46_MultiLineExpressions.po...
Translating pages/38_LoopingOverNestedLists.po...
Translating pages/52_InteractiveProgramsWithInput.po...
Translating pages/28_StringMethodsUnderstandingMutation.po...
Translating pages/2_IntroducingStrings.po...
Translating pages/51_Types.po...
Translating pages/31_EqualsVsIs.po...
Translating pages/21_IntroducingLists.po...
Translating pages/58_IntroducingDictionaries.po...
Translating pages/34_IntroducingFstrings.po...
Translating pages/27_MoreListFunctionsAndMethods.po...
Translating pages/56_GettingElementsAtPositionExercises.po...
Translati

In [16]:
import polib
import os

# Define a function to check if a file is a valid PO file
def is_valid_po_file(file_path):
    try:
        po_file = polib.pofile(file_path)
        return True
    except Exception as e:
        print(f'{file_path} is not a valid PO file: {e}')
        return False

# Define a function to iterate over all generated PO files and check if they are valid
def check_valid_po_files():
    # Iterate over the translated directory
    for root, dirs, files in os.walk('translated'):
        for file in files:
            file_path = os.path.join(root, file)
            is_valid_po_file(file_path)

# Call the function to check the validity of the PO files
check_valid_po_files()


In [18]:
import polib
import os

# Create an empty PO file to store the merged entries
merged_file = polib.POFile()

# Iterate over the translated directory
for root, dirs, files in os.walk('translated'):
    for file in files:
        file_path = os.path.join(root, file)
        # Load each PO file
        po_file = polib.pofile(file_path)
        # Iterate over the entries in the PO file and add them to the merged file
        for entry in po_file:
            merged_file.append(entry)

# Save the merged file
merged_file.save('merged.po')


In [20]:
import polib

# Load the original file
original_file = polib.pofile('futurecoder_English.po')

# Load the merged file
merged_file = polib.pofile('merged.po')

# Count the number of entries in the original file
original_entries_count = len(original_file)

# Count the number of entries in the merged file
merged_entries_count = len(merged_file)

# Print the counts
print(f"Number of entries in the original file: {original_entries_count}")
print(f"Number of entries in the merged file: {merged_entries_count}")


Number of entries in the original file: 1581
Number of entries in the merged file: 1477
